In [ ]:
import openpyxl
from typing import List
import pandas as pd
from enum import Enum
import math
import os
user = os.getlogin()
if user.lower() == "priyr":
    file_path = r"C:\Users\priyr\OneDrive - Oil and Natural Gas Corporation Limited\1. Next Wells\2. Benchmarking\DDKG1 - DDR.xlsm"
elif user.lower() == "priyaranjan":
    file_path = r"C:\Users\Priyaranjan\OneDrive - Oil and Natural Gas Corporation Limited\1. Next Wells\2. Benchmarking\DDKG1 - DDR.xlsm"
else:
    raise Exception("Unknown user, file path not set")

df_rates = pd.read_excel(file_path, sheet_name='Sheet2', engine='openpyxl')
rates = df_rates.iloc[:, 7:10]
rates = rates[~rates['Unnamed: 7'].isna()]
rates.columns = ['Sr No', 'Activity', 'Activity_Rate'] 
def rate(activity):
    row = rates[rates['Activity'] == activity]
    if not row.empty:
        return row['Activity_Rate'].iloc[0]
    else:
        print(f"Activity '{activity}' not found in rates.")
        return None  # or raise an error
activity_rates = dict(zip(rates['Activity'], rates['Activity_Rate']))
# Constants
previous_well_water_depth = 2820 # in meters
riser_jts_previous_well = math.ceil(previous_well_water_depth / 27.4)  # 27.4 m per riser joint
sailing = 280                # NM
water_depth_msl = 2090 #MSL
air_gap = 25
water_depth = water_depth_msl + air_gap
conductor_casing_bml = 80
surface_casing_bml = 630

pipe_od_in = 5.5  # in inches

riser_jts = math.ceil(water_depth / 27.4) 

conductor_casing_depth = water_depth + conductor_casing_bml
surface_casing_depth = water_depth + surface_casing_bml
intermediate_casing_depth = 3118
production_casing_depth = 3925
well_td = 5025

#Coring Depths:
coring_depth_1 = 2915
coring_depth_2 = 3635  
coring_depth_3 = 4275
coring_depth_4 = 4925


string_closed_disp = 0.0964    # bbl/m
string_capacity = 0.0695 # bbl/m
riser_annulus_capacity = 1.1505 - string_closed_disp  # bbl/m

choke_line_vol = 0.0645341 * water_depth  # in bbl, assuming choke line volume is 0.0645341 bbl/m

riser_annulus_vol = riser_annulus_capacity * water_depth  # in bbl
riser_vol = 1.1505 * water_depth

casing_capacity_36_in = 3.47073
casing_capacity_20_in = 1.11804
casing_capacity_13_3_8_in = 0.491207
casing_capacity_9_5_8_in = 0.23215  # 53.5 ppf bbl/m
casing_capacity_7_in = 0.121848   # 29 ppf bbl/m

OH_42_in = 5.621946
OH_26_in = 2.154435
OH_17_5_in = 0.9760052
OH_12_25_in = 0.478242
OH_8_5_in = 0.2302465

casing_annulus_vol_36_in = casing_capacity_36_in - string_closed_disp  # bbl/m
casing_annulus_vol_20_in = casing_capacity_20_in - string_closed_disp   # bbl/m         
casing_annulus_vol_13_3_8_in = casing_capacity_13_3_8_in - string_closed_disp  # bbl/m
casing_annulus_vol_9_5_8_in = casing_capacity_9_5_8_in - string_closed_disp  # bbl/m
casing_annulus_vol_7_in = casing_capacity_7_in - string_closed_disp  # bbl/m

OH_annulus_vol_42_in = OH_42_in - string_closed_disp
OH_annulus_vol_26_in = OH_26_in - string_closed_disp
OH_annulus_vol_17_5_in = OH_17_5_in - string_closed_disp
OH_annulus_vol_12_25_in = OH_12_25_in - string_closed_disp
OH_annulus_vol_8_5_in = OH_8_5_in - string_closed_disp


bottoms_up_17_5_in = riser_annulus_vol + OH_annulus_vol_17_5_in*(intermediate_casing_depth-surface_casing_depth)+casing_annulus_vol_20_in*(surface_casing_depth-water_depth)
bottoms_up_12_25_in = riser_annulus_vol+ OH_annulus_vol_12_25_in*(production_casing_depth-intermediate_casing_depth) + casing_annulus_vol_13_3_8_in*(intermediate_casing_depth-water_depth)
bottoms_up_8_5_in = riser_annulus_vol + OH_annulus_vol_8_5_in*(well_td-production_casing_depth) + casing_annulus_vol_9_5_8_in*(production_casing_depth-water_depth)
def hole_volume_bbl_per_m(diameter_in_inches):
    diameter_m = diameter_in_inches * 0.0254
    volume_m3_per_m = math.pi * (diameter_m / 2) ** 2
    volume_bbl_per_m = volume_m3_per_m / 0.158987294928
    return volume_bbl_per_m
def minimum_flow_rate_bpm(hole_diameter_in, pipe_od_in, v_ft_min=150):
    #v_ft_min = 150  # Using a constant value for minimum velocity
    annular_area_in2 = (math.pi / 4) * (hole_diameter_in**2 - pipe_od_in**2)
    flow_rate_bpm = ((v_ft_min * annular_area_in2) / 1029)  # Convert bbl/min to GPM (1 bbl = 42 gallons)
    return flow_rate_bpm

def minimum_flow_rate_bph(hole_diameter_in, pipe_od_in, v_ft_min=150):
    flow_rate_bph = minimum_flow_rate_bpm(hole_diameter_in, pipe_od_in, v_ft_min)*60    
    return flow_rate_bph
def wiper_trip(bit_depth, shoe_depth, hole_size):
    if hole_size >= 20:
        circulation_volume = ((hole_volume_bbl_per_m(hole_size) - string_closed_disp) * (bit_depth - water_depth) + string_capacity * bit_depth) * 2
        circulation_time = (circulation_volume / (1000 / 42)) / 60  # Assuming pipe OD is 5.5 inches
    else:
        circulation_volume = riser_annulus_vol + (hole_volume_bbl_per_m(hole_size) - string_closed_disp) * (bit_depth - water_depth)
        circulation_time = (circulation_volume / minimum_flow_rate_bpm(hole_size, 5.5)) / 60  # Assuming pipe OD is 5.5 inches

    print(minimum_flow_rate_bpm(hole_size, 5.5))
    print(f"Circulation volume: {circulation_volume:.2f} bbl")
    print(f"Circulation time: {circulation_time:.2f} hours")

    tripping_time = (bit_depth - shoe_depth) * 2 / activity_rates['Tripping_OH']  # Assuming a tripping speed of 30 m/hr
    print(f"Tripping time: {tripping_time:.2f} hours")

    return 1/(circulation_time + tripping_time)

    
def wiper_trip_riserless(bit_depth, shoe_depth, hole_size):
    circulation_volume = ((hole_volume_bbl_per_m(hole_size) - string_closed_disp) * (bit_depth - water_depth) + string_capacity * bit_depth) * 2
    if hole_size >= 20:
        circulation_time = (circulation_volume / (1000 / 42)) / 60  # Assuming pipe OD is 5.5 inches
    else:
        circulation_time = (circulation_volume / minimum_flow_rate_bpm(hole_size, 5.5)) / 60  # Assuming pipe OD is 5.5 inches
    print(minimum_flow_rate_bpm(hole_size, 5.5))
    print(f"Circulation volume: {circulation_volume:.2f} bbl")
    print(f"Circulation time: {circulation_time:.2f} hours")
    tripping_time = (bit_depth - shoe_depth) * 2 / activity_rates['Tripping_OH']  # Assuming a tripping speed of 30 m/hr
    print(f"Tripping time: {tripping_time:.2f} hours")
    return 1/(circulation_time + tripping_time)
def round_trip(bit_depth, shoe_depth, hole_size):
    tripping_time_above_shoe = shoe_depth*2 / activity_rates['Tripping'] 
    print(f"Tripping time above shoe: {tripping_time_above_shoe:.2f} hours")
    wiper_trip_time = wiper_trip(bit_depth, shoe_depth, hole_size)  
    print(f"Wiper trip time: {wiper_trip_time:.2f} hours")    
    return 1/(tripping_time_above_shoe + wiper_trip_time)
def run_and_retrieve_WB():
    bha_make_up_time = activity_rates['M_up_WBRRT_Jet_Sub_BHA']
    tripping_time = 2*water_depth / activity_rates['Tripping']
    set_or_retrieve_time_WB = activity_rates['Jet_WH_Install_WB_Release_WBRRT']
    return 1/(bha_make_up_time + tripping_time + set_or_retrieve_time_WB)
def coring_time(bit_depth, shoe_depth, hole_size, core_no = 1, core_length = 9):
    ''' Circulate (assume to cut one core there is two times circulation for bottoms up)
    Pull out Drilling BHA
    RIH Coring BHA
    Cut Core
    POOH Coring BHA
    RIH Drilling BHA
    '''
    circulation_time = (bottoms_up_12_25_in / minimum_flow_rate_bpm(12.25, pipe_od_in))/60 #Circulate Bottoms Up one extra time for cutting core
    roundtrip_for_BHA = round_trip(bit_depth, shoe_depth, hole_size)  # Assuming this function handles the tripping and wiper trip
    if core_no > 1:
        coring_bha_make_up_time = activity_rates['BHA_Make_Up_Coring_Subsequent'] # Assuming make-up time is the same for each core
    else:
        coring_bha_make_up_time = activity_rates['BHA_Make_Up_Coring']
    trip_in_coring_bha = bit_depth / activity_rates['Tripping']  
    trip_out_coring_bha = bit_depth / activity_rates['Tripping_Coring']  
    coring_time = core_length / activity_rates['Cut_Core']
    coring_trip_total_time = coring_bha_make_up_time + trip_in_coring_bha + coring_time + trip_out_coring_bha
    print(f"Circulation time: {circulation_time:.2f} hours")
    print(f"Roundtrip for BHA: {roundtrip_for_BHA:.2f} hours")
    print(f"Coring time: {coring_time:.2f} hours")      
    print(f"Coring trip total time: {coring_trip_total_time:.2f} hours")
    print(f"coring bha make up time: {coring_bha_make_up_time:.2f} hours")

    return 1/(circulation_time + roundtrip_for_BHA + coring_trip_total_time)
class RigOperationMode(Enum):
    RIH = "Running In"
    POOH = "Pulling Out"
    CIRCULATE = "Circulate"
    ROTATE = "Rotate"
    ROTATE_CIRCULATE = "Rotate & Circulate"
    RIH_ROTATE_CIRCULATE = "RIH + Rotate + Circulate"
    POOH_ROTATE_CIRCULATE = "POOH + Rotate + Circulate"
class MicroTask:
    def __init__(self, name: str, quantity: float, rate: float, mode: RigOperationMode = None):
        self.name = name
        self.quantity = quantity
        self.rate = rate
        self.mode = mode

    @property
    def estimated_time(self) -> float:
        return self.quantity / self.rate if self.rate else 0
class MacroTask:
    def __init__(self, name: str, micro_tasks: List[MicroTask]):
        self.name = name
        self.micro_tasks = micro_tasks

    def total_time(self) -> float:
        return sum(task.estimated_time for task in self.micro_tasks)
class Phase:
    def __init__(self, name: str, macro_tasks: List[MacroTask]):
        self.name = name
        self.macro_tasks = macro_tasks

    def total_time(self) -> float:
        return sum(macro.total_time() for macro in self.macro_tasks)
class DrillingOperation:
    def __init__(self, name: str, phases: List[Phase]):
        self.name = name
        self.phases = phases

    def total_time(self) -> float:
        return sum(phase.total_time() for phase in self.phases)

    def display_plan(self) -> pd.DataFrame:
        rows = []
        for phase in self.phases:
            for macro in phase.macro_tasks:
                for micro in macro.micro_tasks:
                    rows.append({
                        "Phase": phase.name,
                        "Macro Task": macro.name,
                        "Micro Task": micro.name,
                        "Operation Mode": micro.mode.value if micro.mode else "N/A",
                        "Quantity": micro.quantity,
                        "Rate (units/hr)": micro.rate,
                        "Estimated Time (hr)": round(micro.estimated_time, 2)
                    })
        df = pd.DataFrame(rows)
        print(f"\n📘 Subsea Well Drilling Plan: {self.name}")
        print(df)
        print(f"\n🕒 Total Estimated Time: {self.total_time():.2f} hr\n")
        return df


In [ ]:

# Phase 0: BOP & Riser Retrieval / pre rig move
pull_out_mule_shoe = MacroTask("BOP & Riser Retrieval / pre rig move", [
    MicroTask("Pull Out Mule Shoe", previous_well_water_depth+250, rate('Tripping')),
    MicroTask("Lay_Dn_Cement_Head", 1, rate('Lay_Dn_Cement_Head')),
])
pull_out_bop = MacroTask("Pull Out BOP", [
    MicroTask("Rig_up_for_riser_handling", 1, rate('Rig_up_for_riser_handling')),
    MicroTask("Unlatch_BOP", 1, rate('Unlatch_BOP')),
    MicroTask("Secure_Choke_Kill_Boost_Goosenecks", 1, rate('Secure_Choke_Kill_Boost_Goosenecks')),
    MicroTask("L_dn_dummy_riser_n_slip_jt", 1, rate('L_dn_dummy_riser_n_slip_jt')),
    MicroTask("Tripping_BOP", riser_jts_previous_well, rate('Tripping_BOP')),
    MicroTask("Receive_and_Park_BOP", 1, rate('Receive_and_Park_BOP')),
    MicroTask("Sea_Fasten_Prepare_for_rig_move", 1, rate('Sea_Fasten_Prepare_for_rig_move'))
])
sailing_to_well = MacroTask("Sail to Well", [
    MicroTask("Sail to Well", sailing, rate('Sailing'))
])
dp_calibration = MacroTask("DP Calibration", [
    MicroTask("DP Calibration", 1, rate('DP Calibration')) 
])
# Phase 1: Top Hole
investigative_hole = MacroTask("Investigative Hole", [
    MicroTask("Continue RIH 12-1/4'' IH BHA", water_depth/2, rate('Tripping')),
    MicroTask("Drill 12-1/4'' IH Hole", surface_casing_bml, rate('Drilling_IH')), 
    MicroTask("Wiper Trip", 1, wiper_trip_riserless(water_depth+surface_casing_bml, water_depth, 12.25)),
    MicroTask("Circulation", hole_volume_bbl_per_m(12.25)*surface_casing_bml*2, minimum_flow_rate_bph(12.25, 5.5)),
    #MicroTask("Circulation", 0.4782*surface_casing_bml*2, rate('Circulation')),  # This line is commented out as per the context
    MicroTask("POOH 12-1/4'' IH BHA", water_depth+surface_casing_bml, rate('Tripping')),
    MicroTask("Slip and Cut", 1, rate('Slip_and_Cut'))
])

#Drill 42" Hole and run conductor casing
conductor_drilling = MacroTask("Drill 42'' Hole", [
    MicroTask("Make Up MWD BHA", 1, rate('BHA_Make_Up_LWD')),
    MicroTask("Run 26'' x 42'' BHA", water_depth, rate('Tripping')),
    MicroTask("Drill 42''", conductor_casing_bml, rate("Drilling_42_in"), RigOperationMode.ROTATE_CIRCULATE),
    MicroTask("Circulate, POOH, Wait for Soaking & RIH", 1, wiper_trip_riserless(water_depth+conductor_casing_bml, water_depth, 42)+4),
    MicroTask("Circulation", (hole_volume_bbl_per_m(42)*conductor_casing_bml+string_capacity*(conductor_casing_bml+water_depth))*2, 1000/42),
    MicroTask("POOH", conductor_casing_bml+water_depth, rate('Tripping'), RigOperationMode.POOH),
])
conductor_casing = MacroTask("Run Conductor Casing", [
    MicroTask("Change_Pipe_Handler", 1, rate('Change_Pipe_Handler')), 
    MicroTask("Run Conductor Casing", math.ceil(conductor_casing_bml/12), rate('Casing_Lowering_36_in'), RigOperationMode.RIH),
    MicroTask("Install_Ball_Valves_Slope_Indicator_Latch_Mud_Mat", 1, rate('Install_Ball_Valves_Slope_Indicator_Latch_Mud_Mat')),
    MicroTask('Tripping_36_Casing_on_Landing_String', water_depth-surface_casing_bml, rate('Tripping_36_Casing_on_Landing_String')),
    MicroTask('Tripping_36_Casing_on_Landing_String_OH', surface_casing_bml, rate('Tripping_36_Casing_on_Landing_String_OH')),
    MicroTask('Circulation', 1, 1),
    MicroTask("Cement Conductor Casing", 1, rate('36_Casing_Cementation')),
    MicroTask("Wait on Cement", 1, rate('36_Casing_WOC')),
    MicroTask('Release Cart Tool and POOH', water_depth, rate('Tripping')),
    MicroTask('B_Off_Cart_Tool_PO_Inner_String', 1, rate('B_Off_Cart_Tool_PO_Inner_String')),
])
jet_conductor_casing = MacroTask("Jet 36'' Casing", [
    MicroTask("Change_Pipe_Handler", 1, rate('Change_Pipe_Handler')), 
    MicroTask("Run Conductor Casing", math.ceil(conductor_casing_bml/12), rate('Casing_Lowering_36_in'), RigOperationMode.RIH),
    MicroTask("Install_Ball_Valves_Slope_Indicator_Latch_Mud_Mat", 1, rate('Install_Ball_Valves_Slope_Indicator_Latch_Mud_Mat')),
    MicroTask('Tripping_36_Casing_on_Landing_String', water_depth-surface_casing_bml, rate('Tripping_36_Casing_on_Landing_String')),
    MicroTask('Tripping_36_Casing_on_Landing_String_OH', surface_casing_bml, rate('Tripping_36_Casing_on_Landing_String_OH')),
    MicroTask("Jet Conductor Casing ", 1, 9),
    MicroTask("Wait on soaking", 1, 12),
    #To be completed
])

# Phase 2: Surface Section
drill_ahead_surface_drilling = MacroTask("Drill 26'' Hole", [
    #MicroTask("Change_Pipe_Handler", 1, rate('Change_Pipe_Handler')),
    #MicroTask("Make Up Mud Motor, MWD BHA", 1, rate('BHA_Make_Up_LWD')),
    #MicroTask('RIH 26'' BHA', water_depth+conductor_casing_bml-50, rate('Tripping')),
    #MicroTask('RIH with wash down and Tag cement', 30, rate('Tripping_Wash_Dn')),
    #MicroTask('Drill Cement & clear rat hole', 50, rate('Drill_Cement_36_Casing')),
    MicroTask("Drill 26''", surface_casing_bml-conductor_casing_bml, rate('Drilling_26_in'), RigOperationMode.ROTATE_CIRCULATE),
    MicroTask('Wiper Trip', 1, wiper_trip_riserless(surface_casing_depth, conductor_casing_bml, 26)),
    MicroTask("Circulate", (hole_volume_bbl_per_m(26)*surface_casing_bml+string_capacity*surface_casing_depth)*2, 1000/42),
    MicroTask('POOH 26'' BHA', surface_casing_depth, rate('Tripping'), RigOperationMode.POOH),
    MicroTask("Break Off BHA", 1, activity_rates['BHA_Break_Off_LWD']),
])

surface_drilling = MacroTask("Drill 26'' Hole", [
    MicroTask("Change_Pipe_Handler", 1, rate('Change_Pipe_Handler')),
    MicroTask("Make Up Mud Motor, MWD BHA", 1, rate('BHA_Make_Up_LWD')),
    MicroTask('RIH 26'' BHA', water_depth+conductor_casing_bml-50, rate('Tripping')),
    MicroTask('RIH with wash down and Tag cement', 30, rate('Tripping_Wash_Dn')),
    MicroTask('Drill Cement & clear rat hole', 50, rate('Drill_Cement_36_Casing')),
    MicroTask("Drill 26''", surface_casing_bml-conductor_casing_bml, rate('Drilling_26_in'), RigOperationMode.ROTATE_CIRCULATE),
    MicroTask('Wiper Trip', 1, wiper_trip_riserless(surface_casing_depth, conductor_casing_bml, 26)),
    MicroTask("Circulate", (hole_volume_bbl_per_m(26)*surface_casing_bml+string_capacity*surface_casing_depth)*2, (1000/42)*60),
    MicroTask('POOH 26'' BHA', surface_casing_depth, rate('Tripping'), RigOperationMode.POOH),
    MicroTask("Break Off BHA", 1, activity_rates['BHA_Break_Off_LWD']),
])
surface_casing = MacroTask("Run 20'' Casing & Cement", [
    MicroTask('Rig_up_for_Casing', 1, rate('Rig_up_for_Casing')),
    MicroTask('Run_Check_Float_Fun_20_Casing', 1, rate('Run_Check_Float_Fun_20_Casing')),
    MicroTask("Run 20'' Casing", math.ceil(surface_casing_bml/12), rate('Casing_Lowering_20_in'), RigOperationMode.RIH),
    MicroTask('M_Up_HPWHH_Run_Inner_String_Engage_Cart_to_HPWHH', 1, rate('M_Up_HPWHH_Run_Inner_String_Engage_Cart_to_HPWHH')),
    MicroTask('Tripping_Casing_on_Landing_String', water_depth-surface_casing_bml, rate('Tripping_Casing_on_Landing_String')),
    MicroTask('Tripping_Casing_on_Landing_String_OH', surface_casing_bml, rate('Tripping_Casing_on_Landing_String_OH')),
    MicroTask("Cementing 20'' Casing", 1, rate('Cementing_20_in')),
    MicroTask("Wait on Cement", 1, rate('WOC_20_in')),
    MicroTask('Release_Cart_from_HPWHH', 1, rate('Release_Cart_from_HPWHH')),
    MicroTask('POOH Cart Tool to surface', water_depth, rate('Tripping'), RigOperationMode.POOH),
])
bop_lowering = MacroTask("BOP Lowering", [
    MicroTask("Rig_up_for_riser_handling", 1, rate('Rig_up_for_riser_handling')),
    MicroTask('Run_BOP_Splash_Zone', 1, rate('Run_BOP_Splash_Zone')),
    MicroTask('Run BOP on Riser', riser_jts, rate('Tripping_BOP')),
    MicroTask('Pressure Test Choke, Kill, Boost & Conduit Lines', math.ceil(water_depth/500), rate('Pr_Test_C_K_B_C')),
    MicroTask('Run_Slip_Jt_Landing_Jt', 1, rate('Run_Slip_Jt_Landing_Jt')),
    MicroTask('Install_C_K_B_C_Goosenecks', 1, rate('Installl_C_K_B_C_Goosenecks')),
    MicroTask('Pressure Test Choke, Kill & Boost Goosenecks', 1, rate('Pr_Test_C_K_B_C')),
    MicroTask('Install_Saddle_Loops_Engage_MRT', 1, rate('Install_Saddle_Loops_Engare_MRT')),
    MicroTask('Land & Latch BOP. Carry Out Pick up and Slump Test', 1, rate('Land_Latch_BOP_Pick_Up_Test_Slump_Test')),
    MicroTask('Pressure Test Connector. Stroke out Slip Jt & Lay Dn Landing Jt', 1, rate('Pr_Test_Connector_Stroke_Out_Slip_Jt_L_Dn_Landing_Jt')),
    MicroTask('Install_Diverter', 1, rate('Install_Diverter')),
    MicroTask('Rig Down Spider & Gimbal', 1, rate('Rig_Dn_Spider_Gimbal')),
])
# Phase 3: Intermediate Section
intermediate_drilling = MacroTask("Drill 17.5'' Hole", [
    MicroTask('slip_and_cut', 1, rate('Slip_and_Cut')),
    MicroTask('Change_Pipe_Handler', 1, rate('Change_Pipe_Handler')),
    MicroTask('Surface Equipment Pressure Test', 1, rate('Surface_Equipment_Pr_Test')),
    MicroTask('Make up LWD BHA', 1, rate('BHA_Make_Up_LWD')),
    MicroTask('Run 17-1/2'' BHA', water_depth+surface_casing_bml-50, rate('Tripping')),
    MicroTask('BOP Pressure Test', 1, rate('BOP_Pressure_Test')),
    MicroTask('Continue RIH with Wash Dn & Tag Cement', 30, rate('Tripping_Wash_Dn')),
    MicroTask('Drill_Shoe_Track_LOT', 1, rate('Drill_Shoe_Track_LOT')),
    MicroTask('Fingerprint_Flow_Back', 1, rate('Fingerprint_Flow_Back')),
    MicroTask("Drill 17.5''", intermediate_casing_depth-surface_casing_depth, rate('Drilling_17.5_in'), RigOperationMode.ROTATE_CIRCULATE),
    MicroTask('Circulations for Bottoms up (3 times)', bottoms_up_17_5_in*3, minimum_flow_rate_bph(17.5, 5.5)),
    MicroTask('Wiper Trip', 1, wiper_trip(intermediate_casing_depth, surface_casing_depth, 17.5)),
    MicroTask("Circulate", bottoms_up_17_5_in, rate('Circulation')),
    MicroTask("POOH", intermediate_casing_depth, rate('Tripping'), RigOperationMode.POOH),
])
drill_pilot_hole = MacroTask("Drill 12.25'' Pilot Hole", [
    MicroTask('slip_and_cut', 1, rate('Slip_and_Cut')),
    MicroTask('Change_Pipe_Handler', 1, rate('Change_Pipe_Handler')),
    MicroTask('Surface Equipment Pressure Test', 1, rate('Surface_Equipment_Pr_Test')),
    MicroTask('Make up LWD BHA', 1, rate('BHA_Make_Up_LWD')),
    MicroTask('Run 12,25'' BHA', water_depth+surface_casing_bml-50, rate('Tripping')),
    MicroTask('BOP Pressure Test', 1, rate('BOP_Pressure_Test')),
    MicroTask('Continue RIH with Wash Dn & Tag Cement', 30, rate('Tripping_Wash_Dn')),
    MicroTask('Drill_Shoe_Track_LOT', 1, rate('Drill_Shoe_Track_LOT')),
    MicroTask('Fingerprint_Flow_Back', 1, rate('Fingerprint_Flow_Back')),
    MicroTask("Drill 12.25''", intermediate_casing_depth-surface_casing_depth-10, rate('Drilling_12.25_in'), RigOperationMode.ROTATE_CIRCULATE),
    MicroTask(f"Coring at depth {coring_depth_1}", 1, coring_time(coring_depth_1, surface_casing_depth, 12.25, core_no=1, core_length=9)),
    MicroTask("POOH", surface_casing_depth+10, rate('Tripping'), RigOperationMode.POOH),
])
wireline_log_intermediate_casing = MacroTask("Wireline Logging", [
    MicroTask("Basic Logs", 1, 1/3), 
    #MicroTask("Roundtrip for logging", 1, round_trip(intermediate_casing_depth, surface_casing_depth, 17.5)),
])
enlarge_pilot_hole = MacroTask("Enlarge Pilot Hole", [
    MicroTask("Change_Pipe_Handler", 1, rate('Change_Pipe_Handler')),
    MicroTask("Make up MWD BHA", 1, rate('BHA_Make_Up_LWD')),
    MicroTask("Run 17.5'' BHA", surface_casing_depth, rate('Tripping')),
    MicroTask("Enlarge 12.25'' Hole to 17.5''", intermediate_casing_depth-surface_casing_depth, rate('Drilling_17.5_in')*3, RigOperationMode.ROTATE_CIRCULATE),
    MicroTask("Wiper Trip", 1, wiper_trip(intermediate_casing_depth, surface_casing_depth, 17.5)),
    MicroTask("Circulate", bottoms_up_17_5_in, rate('Circulation')),
    MicroTask("POOH", intermediate_casing_depth, rate('Tripping'), RigOperationMode.POOH), 
])    
    
intermediate_casing = MacroTask("Run 13.375'' Casing & Cement", [
    #MicroTask("Roundtrip for Casing Lowering", 1, round_trip(intermediate_casing_depth, surface_casing_depth, 17.5)),
    MicroTask('Retrieve Wear Bush', 1, run_and_retrieve_WB()),
    MicroTask("Rig_up_for_Casing", 1, rate('Rig_up_for_Casing')),
    MicroTask("Run_Check_Float_Fun_13.625_Casing", 1, rate('Run_Check_Float_Fun_13.625_Casing')),
    MicroTask("Run 13.375'' Casing", math.ceil((intermediate_casing_depth-water_depth)/12), rate('Casing_Lowering_13-3/8_in'), RigOperationMode.RIH),
    MicroTask('Tripping_Casing_on_Landing_String', water_depth, rate('Tripping_Casing_on_Landing_String')),
    MicroTask('Land_Latch_Casing_Indexing_Pick_up_test', 1, rate('Land_Latch_Casing_Indexing_Pick_up_test')),
    MicroTask('Cementing_13-3/8_in', 1, rate('Cementing_13-3/8_in')),
    MicroTask('Set_Seal_Assembly', 1, rate('Set_Seal_Assembly')),
    MicroTask('Release_PADPRT_L_Dn_Cement_Head', 1, rate('Release_PADPRT_L_Dn_Cement_Head')),
    MicroTask('POOH PADPRT to surface', water_depth, activity_rates['Tripping']),
    MicroTask('slip_and_cut', 1, rate('Slip_and_Cut')),
    MicroTask('Set Wear Bush at Wellhead', 1, run_and_retrieve_WB()),
])
# Phase 4: Production Section
production_drilling = MacroTask("Drill 12.25'' Hole", [
    MicroTask("Change_Pipe_Handler", 1, rate('Change_Pipe_Handler')),
    MicroTask('Lay_Dn_Cement_Head', 1, rate('Lay_Dn_Cement_Head')),
    MicroTask("Make up LWD BHA", 1, rate('BHA_Make_Up_LWD')),
    MicroTask("Run 12.25'' BHA", intermediate_casing_depth-300, rate('Tripping')),
    MicroTask('BOP_Function_Test', 1, activity_rates['BOP_Function_Test']),
    MicroTask('Choke_Drill', 1, activity_rates['Choke_Drill']),
    MicroTask("Continue RIH with Wash Dn & Tag Cement", 30, rate('Tripping_Wash_Dn')),
    MicroTask("Drill_Shoe_Track_LOT", 1, rate('Drill_Shoe_Track_LOT')),
    MicroTask("Fingerprint_Flow_Back", 1, rate('Fingerprint_Flow_Back')),   
    MicroTask("Drill 12.25''", production_casing_depth-intermediate_casing_depth-10, activity_rates['Drilling_12.25_in'], RigOperationMode.ROTATE_CIRCULATE),
    MicroTask(f"Coring at depth {coring_depth_2}", 1, coring_time(coring_depth_2, intermediate_casing_depth, 12.25, core_no=2, core_length=9)),
    MicroTask("wiper_trip", 1, wiper_trip(production_casing_depth, intermediate_casing_depth, 12.25)),
    MicroTask("Circulate", bottoms_up_12_25_in, minimum_flow_rate_bph(12.25, pipe_od_in)),
    MicroTask("POOH 12.25'' BHA to casing shoe", production_casing_depth-intermediate_casing_depth,  activity_rates['Tripping_Pump_Out'], RigOperationMode.POOH),
    MicroTask("POOH 12.25'' BHA to surface", intermediate_casing_depth-300, activity_rates['Tripping'], RigOperationMode.POOH),
    MicroTask("Break Off BHA", 1, activity_rates['BHA_Break_Off_LWD']),
])

wireline_log_production_casing = MacroTask("Wireline Logging", [
    MicroTask("Basic Logs", 1, 1/3), 
    MicroTask("Roundtrip for logging", 1, round_trip(production_casing_depth, intermediate_casing_depth, 12.25)),
    MicroTask('Advance_Logging', 1, 1/1.5)
])

production_casing = MacroTask("Run 9-5/8'' Casing & Cement", [
    MicroTask('Roundtrip for Casing Lowering', 1, round_trip(production_casing_depth, intermediate_casing_depth, 12.25)),
    MicroTask('Retrieve Wear Bush', 1, run_and_retrieve_WB()),
    MicroTask("Change_Pipe_Handler", 1, rate('Change_Pipe_Handler')),
    MicroTask("Run_Check_Float_Fun_9.625_Casing", 1, rate('Run_Check_Float_Fun_13.625_Casing')),
    MicroTask("Run 9-5/8'' Casing", math.ceil((production_casing_depth-water_depth)/12), rate('Casing_Lowering_13-3/8_in'), RigOperationMode.RIH),
    MicroTask("Tripping_Casing_on_Landing_String", intermediate_casing_depth-(production_casing_depth-water_depth), rate('Tripping_Casing_on_Landing_String')),
    MicroTask("Tripping_Casing_on_Landing_String_OH", production_casing_depth-intermediate_casing_depth, activity_rates['Tripping_Casing_on_Landing_String_OH']),
    MicroTask('Cementing_9-5/8_in', 1, rate('Cementing_13-3/8_in')),
    MicroTask('Set_Seal_Assembly', 1, rate('Set_Seal_Assembly')),
    MicroTask('Release_PADPRT_L_Dn_Cement_Head', 1, rate('Release_PADPRT_L_Dn_Cement_Head')),
    MicroTask('POOH PADPRT to surface', water_depth, activity_rates['Tripping']),
    MicroTask('slip_and_cut', 1, rate('Slip_and_Cut')),
    MicroTask('Set Wear Bush at Wellhead', 1, run_and_retrieve_WB()),
])
# Phase 5: Liner Casing
liner_phase_drilling = MacroTask("Drill 8.5'' Hole", [
    MicroTask("Change_Pipe_Handler", 1, rate('Change_Pipe_Handler')),
    MicroTask("Make up LWD BHA", 1, rate('BHA_Make_Up_LWD')),
    MicroTask("Run 8.5'' BHA", production_casing_depth-300, rate('Tripping')),
    MicroTask('BOP_Pressure_Test', 1, activity_rates['BOP_Pressure_Test']),
    MicroTask('Choke_Drill', 1, activity_rates['Choke_Drill']),
    MicroTask("Continue RIH with Wash Dn & Tag Cement", 30, rate('Tripping_Wash_Dn')),
    MicroTask("Drill_Shoe_Track_LOT", 1, rate('Drill_Shoe_Track_LOT')),
    MicroTask("Fingerprint_Flow_Back", 1, rate('Fingerprint_Flow_Back')),   
    MicroTask("Drill 8.5''", well_td-production_casing_depth-10, activity_rates['Drilling_12.25_in'], RigOperationMode.ROTATE_CIRCULATE),
    MicroTask(f"Coring at depth {coring_depth_3}", 1, coring_time(coring_depth_3, production_casing_depth, 8.5, core_no=3, core_length=9)),
    MicroTask(f"Coring at depth {coring_depth_4}", 1, coring_time(coring_depth_4, production_casing_depth, 8.5, core_no=4, core_length=9)),
    MicroTask("wiper_trip", 1, wiper_trip(well_td, production_casing_depth, 8.5)),
    MicroTask("Circulate", bottoms_up_8_5_in, minimum_flow_rate_bph(8.5, pipe_od_in)),
    MicroTask("POOH 12.25'' BHA to casing shoe", well_td - production_casing_depth,  activity_rates['Tripping_Pump_Out'], RigOperationMode.POOH),
    MicroTask("POOH 12.25'' BHA to surface", production_casing_depth-300, activity_rates['Tripping'], RigOperationMode.POOH),
    MicroTask("Break Off BHA", 1, activity_rates['BHA_Break_Off_LWD']),
])

wireline_log_liner_casing = MacroTask("Wireline Logging", [
    MicroTask("Basic Logs", 1, rate('Logging')), 
    MicroTask("Roundtrip for logging", 1, round_trip(well_td, production_casing_depth, 8.5)),
    MicroTask('Advance_Logging', 1, activity_rates['Logging_MDT'])
])

production_liner = MacroTask("Run 7'' Liner & Cement", [
    MicroTask('Roundtrip for Casing Lowering', 1, round_trip(well_td, production_casing_depth, 8.5)),
    MicroTask("Change_Pipe_Handler", 1, rate('Change_Pipe_Handler')),
    MicroTask("Run_Check_Float_Fun_7''_Liner", 1, rate('Run_Check_Float_Fun_13.625_Casing')),
    MicroTask("Run 7'' Liner", math.ceil((well_td - production_casing_depth + 150)/12), rate('Casing_Lowering_13-3/8_in'), RigOperationMode.RIH),
    MicroTask("Tripping_Casing_on_Landing_String", production_casing_depth-(well_td-production_casing_depth+150), rate('Tripping_Casing_on_Landing_String')),
    MicroTask("Tripping_Casing_on_Landing_String_OH", well_td - production_casing_depth, activity_rates['Tripping_Casing_on_Landing_String_OH']),
    MicroTask('Cementing_7'' Liner', 1, rate('Cementing_13-3/8_in')),
    MicroTask('Set_Liner_Hanger_Seal', 1, rate('Set_Seal_Assembly')),
    MicroTask('Release_Liner_Hanger_Running_Tool', 1, rate('Release_PADPRT_L_Dn_Cement_Head')),
    MicroTask('POOH Liner Hanger Running Tool to surface', water_depth, activity_rates['Tripping']),
    MicroTask('slip_and_cut', 1, rate('Slip_and_Cut')),
])

#Phase 6: Completion Preparation
completion_prep = MacroTask("Scraping, CBL-VDL & Hermetical", [
    MicroTask("RIH Tandem Scraper from surface to Scraping depth", production_casing_depth-500, activity_rates['Tripping'], RigOperationMode.RIH),
    MicroTask('Scrape Casing', well_td-production_casing_depth+500, activity_rates['Tripping_Pump_Out']),
    MicroTask('POOH Tandem Scraping BHA', well_td, activity_rates['Tripping']), 
    MicroTask('Record CBL VDL Log', 1, activity_rates['Logging']*5),
    MicroTask('Run Mule Shoe for hermetical and infllow test', well_td, activity_rates['Tripping']),
    MicroTask('Changeover Well Volume to from WBM to sea water', bottoms_up_8_5_in, minimum_flow_rate_bph(8.5, 5)),
    MicroTask('Carry out Inflow & Hermetical Test', 1, 7), 
])
# Define Phases
phase0 = Phase("Rig move", [pull_out_mule_shoe, pull_out_bop, sailing_to_well, dp_calibration])
phase1 = Phase("Top Hole", [investigative_hole, conductor_drilling, conductor_casing, surface_drilling, surface_casing, bop_lowering])
#phase1 = Phase("Top Hole", [investigative_hole, jet_conductor_casing, drill_ahead_surface_drilling,  surface_casing, bop_lowering])
phase2 = Phase("Intermediate Section", [drill_pilot_hole, wireline_log_intermediate_casing, enlarge_pilot_hole, intermediate_casing])
phase3 = Phase("Production Casing", [production_drilling, wireline_log_production_casing, production_casing])
phase4 = Phase("Production Liner", [liner_phase_drilling, wireline_log_liner_casing, production_liner])
phase5 = Phase("Hermetical & Inflow Test", [completion_prep])
# Create Operation
subsea_well = DrillingOperation("Subsea Well A", [phase0, phase1, phase2, phase3, phase4, phase5])
#subsea_well = DrillingOperation("Subsea Well A", [phase0, phase1, phase4, phase5])
df = subsea_well.display_plan()

In [218]:
df['Plan_Days'] = df['Estimated Time (hr)']/24
df['Cum_Days'] = df['Plan_Days'].cumsum()

In [220]:
output_file = r"C:\Users\priyr\OneDrive - Oil and Natural Gas Corporation Limited\1. Next Wells\2. Benchmarking\Subsea_Well_Drilling_Plan.xlsx"
df.to_excel(output_file, index=False)

In [192]:

rates

,Sr No,Activity,Activity_Rate
0,1.0,36_Casing_Cementation,0.333333
1,2.0,36_Casing_WOC,0.055556
2,3.0,42_Hole_RIH_Displace,82.909091
3,4.0,42_Hole_Soaking,0.250000
4,5.0,B_Off_Cart_Tool_PO_Inner_String,0.400000
...,...,...,...
83,84.0,WB_Retrival,1.190476
84,85.0,Wiper_Trip,114.666667
85,86.0,WOC_20_in,0.083333
86,87.0,WOC_Bottom_Plug,0.055556


In [ ]:
pd.set_option('display.max_rows', None)  # Show all rows in the DataFrame

In [200]:
for phase in subsea_well.phases:
    print(f"{phase.name}: {phase.total_time()/24:.2f} days")

Rig move: 6.00 days
Top Hole: 18.30 days
Intermediate Section: 9.58 days
Production Casing: 12.68 days
Production Liner: 25.15 days
Hermetical & Inflow Test: 3.13 days


In [117]:
def rate_no(no):
    row = rates[rates['Sr No'] == no]
    if not row.empty:
        return row['Activity_Rate'].iloc[0], row['Activity'].iloc[0]
    else:
        print(f"Activity '{no}' not found in rates.")
        return None  # or raise an error

In [118]:
rate_no(35)

(np.float64(0.535714285683103),
 'Install_Ball_Valves_Slope_Indicator_Latch_Mud_Mat')